In [ ]:
import pandas as pdgo
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib.patches import Patch

#**1. DATA UNDERSTANDING**

**Keterangan Variabel**

* Y  : Rata-rata pengeluaran untuk rokok dan tembakau per minggu (Ribu rupiah)
* X1 : Persentase Pengeluaran per Kapita Ribu rupiah (%)
* X2 : PDRB perkapita (Ribu rupiah)
* X3 : Jumlah Penduduk (Ribu jiwa)
* X4 : Jumlah Penduduk Miskin (Ribu rupiah)
* X5 : Rata-Rata Lama Sekolah (Tahun)


In [ ]:
data = pd.read_excel("../data/Data_Pelatihan.xlsx")
data

In [ ]:
data.head(5)

In [ ]:
data.tail(5)

In [ ]:
data.shape

In [ ]:
data.dtypes

#**2. DATA CLEANING**

### MEMERIKSA MISSING VALUES

In [ ]:
data.isnull().sum()

### MEMERIKSA NILAI DUPLIKAT

In [ ]:
print("\nJumlah baris duplikat:")
print(data
      .duplicated().sum())

#**3. EXPLORATORY DATA ANALYSIS (EDA)**

In [ ]:
data.describe()

In [ ]:
numerical_col = ["Y","X1","X2","X3","X4","X5"]

In [ ]:
plt.figure(figsize=(15, 15))
for i, column in enumerate(numerical_col, start=1):
    ax = plt.subplot(3, 3, i)
    vals = data[column].dropna()
    sns.histplot(
        vals,
        bins=30,
        kde=False,
        color="blue",
        edgecolor="orange",
        alpha= 0.8,
        stat="density"
    )
    sns.kdeplot(
        vals,
        color="orange",
        linewidth=2
    )

    ax.set_title(f"Histogram {column}")
    ax.set_xlabel(column)
plt.tight_layout()
plt.show()

In [ ]:
num_variables = len(numerical_col)
num_rows = 2
num_cols = (num_variables + num_rows - 1) // num_rows

plt.figure(figsize=(15, 8))

for i, variable in enumerate(numerical_col):
    plt.subplot(num_rows, num_cols, i+1)
    plt.boxplot(data[variable], whis=1.5)
    plt.title(variable)

plt.tight_layout()
plt.show()

In [ ]:
sns.set_theme(style="whitegrid", palette="Spectral")
plt.figure(figsize=(18, 12))
for i, column in enumerate(numerical_col, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(
        x="Tahun",
        y=column,
        data=data,
        palette="Spectral",
        showfliers=False
    )
    sns.stripplot(
        x="Tahun",
        y=column,
        data=data,
        color="black",
        jitter=True,
        alpha=0.4,
        size=3
    )

    means = data.groupby("Tahun")[column].mean()
    for j, year in enumerate(sorted(data["Tahun"].unique())):
        plt.scatter(
            j, means.loc[year],
            color="red",
            s=80,
            marker="D",
            label="Mean" if i == 1 and j == 0 else ""
        )

    plt.title(f"{column}", fontsize=14)
    plt.xlabel("Tahun", fontsize=10)
    plt.ylabel(column, fontsize=10)

# Tambahkan legenda untuk mean sekali saja
plt.legend(loc="upper right", bbox_to_anchor=(1.2, 1))

plt.suptitle("", fontsize=18, weight="bold")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
cols = [c for c in ["Y","X1","X2","X3","X4","X5"] if c in data.columns]
plt.figure(figsize=(8,7))
sns.heatmap(data[cols].corr(), cmap="bwr", annot=True, fmt=".2f", linewidths=.2)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
java_provinces = [
    "Banten",
    "DKI Jakarta",
    "Jawa Barat",
    "Jawa Tengah",
    "Daerah Istimewa Yogyakarta",
    "Jawa Timur",
]

df_java_2024 = (
    data.query("Tahun == 2024 and Provinsi in @java_provinces")
         .copy()
)
prov_rank = (df_java_2024
             .groupby("Provinsi", as_index=False)["Y"]
             .mean()
             .sort_values("Y", ascending=False))

top_province = prov_rank.iloc[0, 0]
top_value = prov_rank.iloc[0, 1]

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=prov_rank,
    x="Y", y="Provinsi",
    palette="mako"
)
plt.title("Rata-rata Pengeluaran Rokok/Tembakau (Y)\nProvinsi di Pulau Jawa – 2024", fontsize=14, weight="bold")
plt.xlabel("Rata-rata Y (Ribu Rp/minggu)")
plt.ylabel("Provinsi")

for p in ax.patches:
    w = p.get_width()
    y = p.get_y() + p.get_height()/2
    ax.text(w + (0.01*prov_rank["Y"].max()), y, f"{w:,.0f}", va="center", fontsize=10)

for p in ax.patches:
    if p.get_y() + p.get_height()/2 in ax.get_yticks():
        label = ax.get_yticklabels()[list(ax.get_yticks()).index(p.get_y() + p.get_height()/2)].get_text()
        if label == top_province:
            p.set_edgecolor("orange")
            p.set_linewidth(2.5)

plt.tight_layout()
plt.show()

In [ ]:
import geopandas as gpd
# --- Load geojson ---
geo_path = "../data/Indonesia_Province.json"
gdf = gpd.read_file(geo_path)

# --- Tampilkan semua kolom ---
print("\n=== Kolom tersedia di geojson ===")
print(gdf.columns.tolist())

# --- Cek isi 5 baris pertama ---
print("\n=== 5 baris pertama ===")
print(gdf.head())

# --- Cek kolom string dan 10 nilai unik pertama ---
print("\n=== Nilai unik dari kolom string ===")
for c in gdf.columns:
    if gdf[c].dtype == "object":
        print(f"\nKolom: {c}")
        print(gdf[c].unique()[:10])

In [ ]:
# --- 1) Import ---
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# --- 2) Data rata-rata Y per provinsi ---
java_provinces = [
    "Banten",
    "DKI Jakarta",
    "Jawa Barat",
    "Jawa Tengah",
    "Daerah Istimewa Yogyakarta",
    "Jawa Timur",
]

# fungsi normalisasi nama provinsi
def norm_prov(s: str) -> str:
    s = str(s).strip().upper()
    if "YOGYAKARTA" in s:
        return "Daerah Istimewa Yogyakarta"
    if "JAKARTA" in s or "DKI" in s:
        return "DKI Jakarta"
    if "BANTEN" in s:
        return "Banten"
    if "JAWA BARAT" in s:
        return "Jawa Barat"
    if "JAWA TENGAH" in s:
        return "Jawa Tengah"
    if "JAWA TIMUR" in s:
        return "Jawa Timur"
    return s.title()

# hitung rata-rata Y provinsi Jawa
df_java_2024 = data.query("Tahun == 2024 and Provinsi in @java_provinces").copy()
prov_rank = (df_java_2024.groupby("Provinsi", as_index=False)["Y"]
             .mean().sort_values("Y", ascending=False))

# normalisasi nama di prov_rank juga
prov_rank["Provinsi"] = prov_rank["Provinsi"].map(norm_prov)

top_province = prov_rank.iloc[0, 0]
top_value    = prov_rank.iloc[0, 1]

# --- 3) Load geojson ---
geo_path = "../data/Indonesia_Province.json"
gdf = gpd.read_file(geo_path)

# pakai kolom Propinsi dan normalisasi
gdf["Provinsi"] = gdf["Propinsi"].map(norm_prov)

# --- 4) Ambil provinsi Jawa dan merge nilai Y ---
gdf_java = gdf[gdf["Provinsi"].isin(java_provinces)].copy()
gdf_plot = gdf_java.merge(prov_rank, on="Provinsi", how="left")

# --- 5) Plot choropleth ---
fig, ax = plt.subplots(figsize=(8, 5))

gdf_plot.plot(
    column="Y",
    cmap="Blues",
    linewidth=0.8,
    edgecolor="white",
    legend=True,
    legend_kwds={"label": "Rata-rata Y (Ribu Rp/minggu)", "shrink": 0.6},
    ax=ax
)

# highlight provinsi tertinggi
subset = gdf_plot[gdf_plot["Provinsi"] == top_province]
if not subset.empty:
    subset.boundary.plot(ax=ax, color="orange", linewidth=2.5)

# label nilai di representative point
for _, r in gdf_plot.iterrows():
    if r["geometry"] is None or r["geometry"].is_empty:
        continue
    pt = r.geometry.representative_point()
    ax.text(pt.x, pt.y, f"{r['Y']:.0f}", ha="center", va="center",
            fontsize=9, weight="bold")

# judul & anotasi
ax.set_title(
    "Rata-rata Pengeluaran Rokok/Tembakau (Y)\nProvinsi di Pulau Jawa – 2024",
    fontsize=14, weight="bold"
)
ax.set_axis_off()

ax.annotate(
    f"Teratas: {top_province} ({top_value:,.0f})",
    xy=(0.01, 0.02), xycoords="figure fraction",
    fontsize=10, bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="orange", lw=1.5)
)

plt.tight_layout()
plt.show()

In [ ]:
df_top = df_java_2024.query("Provinsi == @top_province")
kabkot_rank = (df_top
               .groupby(["Provinsi", "Kabupaten/Kota"], as_index=False)["Y"]
               .mean()
               .sort_values("Y", ascending=False))
plt.figure(figsize=(10, max(6, 0.35*len(kabkot_rank))))
ax2 = sns.barplot(
    data=kabkot_rank,
    x="Y", y="Kabupaten/Kota",
    palette="rocket"
)
plt.title(f"Rata-rata Pengeluaran Rokok/Tembakau (Y)\nKab/Kota di {top_province} – 2024", fontsize=14, weight="bold")
plt.xlabel("Rata-rata Y (Ribu Rp/minggu)")
plt.ylabel("Kabupaten/Kota")

for p in ax2.patches:
    w = p.get_width()
    y = p.get_y() + p.get_height()/2
    ax2.text(w + (0.01*kabkot_rank["Y"].max()), y, f"{w:,.0f}", va="center", fontsize=9)

plt.tight_layout()
plt.show()

#**4. MODELLING & EVALUATION**

In [ ]:
# Pisahkan fitur (variabel X) & target (variabel y)
X = data[["X1","X2","X3","X4","X5"]]
y = data["Y"].astype(float)

In [ ]:
# splitting data 80% data training dan 20% data testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
#STANDARISASI
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled  = scaler.transform(X_test)

In [ ]:
# NORMALISASI
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

## **METODE RANDOM FOREST**

In [ ]:
# === 1) Model Random Forest ===
model = RandomForestRegressor(
    n_estimators=1000,
    random_state=42,
    n_jobs=-1,
    max_features="sqrt"
)
model.fit(X_train_scaled, y_train)

# === 2) Prediksi & Evaluasi ===
y_pred = model.predict(X_test_scaled)

mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"MAE : {mae:,.2f}")
print(f"R²  : {r2:.4f}")

# === 3) Visual: Prediksi vs Aktual (Scatter + garis 45°) ===
hasil = pd.DataFrame({"Aktual": y_test.values, "Prediksi": y_pred}, index=X_test.index)

plt.figure(figsize=(6,5))
plt.scatter(hasil["Aktual"], hasil["Prediksi"], s=35, alpha=0.75)
lo = min(hasil["Aktual"].min(), hasil["Prediksi"].min())
hi = max(hasil["Aktual"].max(), hasil["Prediksi"].max())
plt.plot([lo, hi], [lo, hi], color="orange", linewidth=2)  # garis ideal
plt.title("Prediksi vs Aktual (Test) – Random Forest")
plt.xlabel("Aktual Y"); plt.ylabel("Prediksi Y")
plt.grid(True, alpha=.3)
plt.tight_layout()
plt.show()


In [ ]:
feature_names = list(X.columns)

# --- 1) Ambil Feature Importance dari model Random Forest ---
# Atribut .feature_importances_ berisi nilai MDI (Mean Decrease in Impurity)
importances = model.feature_importances_

# --- 2) Buat tabel ringkas menggunakan pandas Series ---
imp_series = pd.Series(importances, index=feature_names)
imp_table = pd.DataFrame({
    "Importance": imp_series
}).sort_values("Importance", ascending=False)

print("Tabel Feature Importance:")
print(imp_table)
print("-" * 30)


# --- 3) Plot horizontal (lebih informatif) ---
# Urutkan nilai agar plot lebih rapi (dari terkecil ke terbesar)
imp_plot = imp_series.sort_values(ascending=True)

plt.figure(figsize=(8, 5))
bars = plt.barh(imp_plot.index, imp_plot.values, color="darkcyan", edgecolor="black")

# Menambahkan label nilai di ujung setiap bar
for bar in bars:
    width = bar.get_width()
    plt.text(width * 1.01,  # Posisi x sedikit di kanan bar
             bar.get_y() + bar.get_height() / 2, # Posisi y di tengah bar
             f'{width:.4f}', # Format teks
             ha='left',
             va='center',
             fontsize=9)

plt.title("Random Forest Feature Importance (MDI)")
plt.xlabel("Importance (Mean Decrease in Impurity)")
plt.grid(True, axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# === MENU INPUT X1–X5 → PREDIKSI Y (Jupyter) ===
# Jika ipywidgets belum ada, jalankan:  # !pip install ipywidgets
import ipywidgets as w
from IPython.display import display

feature_names = ["X1","X2","X3","X4","X5"]

# Ringkasan rentang & median berdasarkan data latih (membantu mengisi nilai yang masuk akal)
mins = X_train[feature_names].min()
maxs = X_train[feature_names].max()
meds = X_train[feature_names].median()

# Widget input (diinisialisasi ke median pelatihan)
w_X1 = w.FloatText(value=float(meds["X1"]), description="X1", layout=w.Layout(width="280px"))
w_X2 = w.FloatText(value=float(meds["X2"]), description="X2", layout=w.Layout(width="280px"))
w_X3 = w.FloatText(value=float(meds["X3"]), description="X3", layout=w.Layout(width="280px"))
w_X4 = w.FloatText(value=float(meds["X4"]), description="X4", layout=w.Layout(width="280px"))
w_X5 = w.FloatText(value=float(meds["X5"]), description="X5", layout=w.Layout(width="280px"))
inputs = [w_X1, w_X2, w_X3, w_X4, w_X5]

btn_predict = w.Button(description="Prediksi Y", button_style="success", icon="calculator")
btn_reset   = w.Button(description="Reset ke Median", icon="refresh")
out = w.Output()

def _prediksi(_=None):
    # Kumpulkan nilai, scaling dengan scaler yg sudah fit, lalu prediksi dengan model
    vals = [[w_X1.value, w_X2.value, w_X3.value, w_X4.value, w_X5.value]]
    df_new = pd.DataFrame(vals, columns=feature_names)
    X_new_scaled = scaler.transform(df_new)
    y_hat = float(model.predict(X_new_scaled)[0])

    with out:
        out.clear_output()
        display(pd.DataFrame({
            "Fitur": feature_names,
            "Nilai": [w.value for w in inputs],
            "Min(latih)": [mins[f] for f in feature_names],
            "Max(latih)": [maxs[f] for f in feature_names],
        }))
        print(f"Perkiraan Y: {y_hat:,.2f} (ribu Rp/minggu)")

def _reset(_=None):
    for f, wdg in zip(feature_names, inputs):
        wdg.value = float(meds[f])
    with out:
        out.clear_output()

btn_predict.on_click(_prediksi)
btn_reset.on_click(_reset)

title = w.HTML("<h3>Menu Prediksi Y</h3><p>Isi X1–X5 lalu klik <b>Prediksi Y</b>. "
               "Rentang yang ditampilkan berasal dari data latih.</p>")

ui = w.VBox([
    title,
    w.HBox([w_X1, w_X2]),
    w.HBox([w_X3, w_X4]),
    w.HBox([w_X5]),
    w.HBox([btn_predict, btn_reset]),
    out
])

display(ui)

##**METODE XGBOOST**

In [ ]:
# === 1) Model XGBoost ===
xgb = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
    tree_method="hist"   # cepat
)
xgb.fit(X_train_scaled, y_train)

# === 2) Prediksi & Evaluasi ===
y_pred = xgb.predict(X_test_scaled)

mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"MAE : {mae:,.2f}")
print(f"R²  : {r2:.4f}")

# === 3) Visual: Prediksi vs Aktual (Scatter + garis 45°) ===
hasil = pd.DataFrame({"Aktual": y_test.values, "Prediksi": y_pred}, index=X_test.index)

plt.figure(figsize=(6,5))
plt.scatter(hasil["Aktual"], hasil["Prediksi"], s=35, alpha=0.75)
lo = min(hasil["Aktual"].min(), hasil["Prediksi"].min())
hi = max(hasil["Aktual"].max(), hasil["Prediksi"].max())
plt.plot([lo, hi], [lo, hi], color="orange", linewidth=2)  # garis ideal
plt.title("Prediksi vs Aktual (Test) – XGBoost")
plt.xlabel("Aktual Y"); plt.ylabel("Prediksi Y")
plt.grid(True, alpha=.3)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# --- nama fitur asli (sebelum scaling) ---
feature_names = list(X.columns)  # ["X1","X2","X3","X4","X5"]

# --- 1) Importance versi 'gain' (lebih informatif) ---
gain_dict = xgb.get_booster().get_score(importance_type="gain")  # keys: 'f0','f1',...
gain_series = pd.Series({feature_names[int(k[1:])]: v for k, v in gain_dict.items()})
gain_series = gain_series.reindex(feature_names).fillna(0.0)

# --- 2) (Opsional) Importance bawaan (split frequency) ---
split_series = pd.Series(xgb.feature_importances_, index=feature_names)

# Tabel ringkas
imp_table = pd.DataFrame({
    "Gain": gain_series,
    "Split": split_series
}).sort_values("Gain", ascending=False)
print(imp_table)

# --- Plot horizontal (pakai 'Gain') ---
imp_plot = gain_series.sort_values(ascending=True)
plt.figure(figsize=(6,4))
plt.barh(imp_plot.index, imp_plot.values, color="darkblue", edgecolor="orange")
for i, v in enumerate(imp_plot.values):
    plt.text(v, i, f"{v:.3f}", va="center", ha="left", fontsize=9)
plt.title("XGBoost Feature Importance (Gain)")
plt.xlabel("Importance (Gain)")
plt.tight_layout()
plt.show()

In [ ]:
# === MENU INPUT X1–X5 → PREDIKSI Y (Jupyter) ===
# Jika ipywidgets belum ada, jalankan:  # !pip install ipywidgets
import ipywidgets as w
from IPython.display import display

feature_names = ["X1","X2","X3","X4","X5"]

# Ringkasan rentang & median berdasarkan data latih (membantu mengisi nilai yang masuk akal)
mins = X_train[feature_names].min()
maxs = X_train[feature_names].max()
meds = X_train[feature_names].median()

# Widget input (diinisialisasi ke median pelatihan)
w_X1 = w.FloatText(value=float(meds["X1"]), description="X1", layout=w.Layout(width="280px"))
w_X2 = w.FloatText(value=float(meds["X2"]), description="X2", layout=w.Layout(width="280px"))
w_X3 = w.FloatText(value=float(meds["X3"]), description="X3", layout=w.Layout(width="280px"))
w_X4 = w.FloatText(value=float(meds["X4"]), description="X4", layout=w.Layout(width="280px"))
w_X5 = w.FloatText(value=float(meds["X5"]), description="X5", layout=w.Layout(width="280px"))
inputs = [w_X1, w_X2, w_X3, w_X4, w_X5]

btn_predict = w.Button(description="Prediksi Y", button_style="success", icon="calculator")
btn_reset   = w.Button(description="Reset ke Median", icon="refresh")
out = w.Output()

def _prediksi(_=None):
    # Kumpulkan nilai, scaling dengan scaler yg sudah fit, lalu prediksi dengan model
    vals = [[w_X1.value, w_X2.value, w_X3.value, w_X4.value, w_X5.value]]
    df_new = pd.DataFrame(vals, columns=feature_names)
    X_new_scaled = scaler.transform(df_new)
    y_hat = float(xgb.predict(X_new_scaled)[0])

    with out:
        out.clear_output()
        display(pd.DataFrame({
            "Fitur": feature_names,
            "Nilai": [w.value for w in inputs],
            "Min(latih)": [mins[f] for f in feature_names],
            "Max(latih)": [maxs[f] for f in feature_names],
        }))
        print(f"Perkiraan Y: {y_hat:,.2f} (ribu Rp/minggu)")

def _reset(_=None):
    for f, wdg in zip(feature_names, inputs):
        wdg.value = float(meds[f])
    with out:
        out.clear_output()

btn_predict.on_click(_prediksi)
btn_reset.on_click(_reset)

title = w.HTML("<h3>Menu Prediksi Y</h3><p>Isi X1–X5 lalu klik <b>Prediksi Y</b>. "
               "Rentang yang ditampilkan berasal dari data latih.</p>")

ui = w.VBox([
    title,
    w.HBox([w_X1, w_X2]),
    w.HBox([w_X3, w_X4]),
    w.HBox([w_X5]),
    w.HBox([btn_predict, btn_reset]),
    out
])

display(ui)

# **5. CLUSTERING**


## **K-MEANS CLUSTERING**

In [ ]:
#!pip install "numpy<2" # di run cukup sekali, setelah itu mulai dari upload dataset dan run sintaks dari panggil package dan import data

In [ ]:
#!pip install scikit-learn-extra

In [ ]:
x_train = data[["Y"]].values

In [ ]:
# Elbow method
from sklearn.cluster import KMeans
cs = []
for i in range (1,10):
    kmeans = KMeans(n_clusters=i, init = 'k-means++',max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(x_train)
    cs.append(kmeans.inertia_)
plt.plot(range(1,10), cs)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('CS')
plt.show()

In [ ]:
# Silhouette method
from sklearn.metrics import silhouette_score
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters, max_iter=300)
    kmeans.fit(x_train)
    cluster_labels = kmeans.labels_
    silhouette_avg = silhouette_score(x_train, cluster_labels)
    print("For n_cluster={0}, the silhoutte score is {1}".format(num_clusters, silhouette_avg))

In [ ]:
# didapat Jumlah k = 2 Terbaik
kmeans = KMeans(n_clusters = 2, random_state = 0)
kmeans

In [ ]:
y_cluster = kmeans.fit_predict(x_train)

In [ ]:
data['kelas'] = y_cluster

In [ ]:
kmeans.cluster_centers_

In [ ]:
data["kelas"] = data["kelas"].astype("object")

data.loc[data.kelas==0, "kelas"] = "Rendah"
data.loc[data.kelas==1, "kelas"] = "Tinggi"

In [ ]:
data.groupby(["kelas"]).count()

In [ ]:
colors = {'Tinggi': 'red', 'Rendah': 'black'}
plt.figure(figsize=(12, 6))
sns.scatterplot(
    data=data,
    x=data.index,
    y='Y',
    hue='kelas',
    palette=colors,
    s=25,
    alpha=0.7
)
plt.title('Scatter Plot Kluster Berdasarkan Nilai Y')
plt.xlabel('Indeks Data')
plt.ylabel('Nilai Y')
plt.legend(title='Kelas')
plt.grid(True, linestyle='--')
plt.show()

## **K-MEDOIDS**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
from sklearn_extra.cluster import KMedoids

x_train = data[["Y"]].values

In [ ]:
# --- Elbow method (menggunakan K-Medoids) ---
cs = []
for i in range (1,10):
    # Menggunakan KMedoids sebagai ganti KMeans
    kmedoids = KMedoids(n_clusters=i, random_state = 0) # <-- DIUBAH
    kmedoids.fit(x_train)
    # K-Medoids tidak punya atribut .inertia_, tapi .inertia_ K-Means
    # adalah jumlah kuadrat jarak. Hal yang setara di K-Medoids adalah .inertia_ juga
    # (walaupun dihitung sedikit berbeda, tujuannya sama).
    cs.append(kmedoids.inertia_)

plt.plot(range(1,10), cs)
plt.title('The Elbow Method for K-Medoids')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

In [ ]:
# --- Silhouette method (menggunakan K-Medoids) ---
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for num_clusters in range_n_clusters:
    # Menggunakan KMedoids sebagai ganti KMeans
    kmedoids = KMedoids(n_clusters=num_clusters, random_state=0) # <-- DIUBAH
    kmedoids.fit(x_train)
    cluster_labels = kmedoids.labels_

    silhouette_avg = silhouette_score(x_train, cluster_labels)
    print("For n_cluster={0}, the silhoutte score is {1}".format(num_clusters, silhouette_avg))

In [ ]:
# --- Terapkan model dengan k terbaik ---
kmedoids = KMedoids(n_clusters = 2, random_state = 0) # <-- DIUBAH
y_cluster2 = kmedoids.fit_predict(x_train)

In [ ]:
data['kelas_2'] = y_cluster2
print("\nPusat Cluster (Medoids):")
print(kmedoids.cluster_centers_) # <-- Ini adalah data point asli

In [ ]:
data["kelas_2"] = data["kelas_2"].astype("object")

data.loc[data.kelas_2==0, "kelas_2"] = "Rendah"
data.loc[data.kelas_2==1, "kelas_2"] = "Tinggi"

In [ ]:
data.groupby(["kelas_2"]).count()

In [ ]:
colors = {'Tinggi': 'red', 'Rendah': 'black'}
plt.figure(figsize=(12, 6))
sns.scatterplot(
    data=data,
    x=data.index,
    y='Y',
    hue='kelas_2',
    palette=colors,
    s=25,
    alpha=0.7
)
plt.title('Scatter Plot Kluster K-Medoids')
plt.xlabel('Indeks Data')
plt.ylabel('Nilai Y')
plt.legend(title='Kelas')
plt.grid(True, linestyle='--')
plt.show()

In [ ]:
data

# **6. PETA HASIL CLUSTERING**

In [ ]:
gdf["Provinsi"] = gdf["Propinsi"].map(norm_prov)
gdf_java = gdf[gdf["Provinsi"].isin(java_provinces)].to_crs(4326).copy()

# --- 4) Ambil label kelas per provinsi (pakai modus) ---
kelas_1 = data.groupby("Provinsi")["kelas"].agg(lambda x: x.mode()[0]).reset_index(name="kelas_1")
kelas_2 = data.groupby("Provinsi")["kelas_2"].agg(lambda x: x.mode()[0]).reset_index(name="kelas_2")
gdf_java = gdf_java.merge(kelas_1, on="Provinsi", how="left").merge(kelas_2, on="Provinsi", how="left")

# --- 5) Pemetaan warna ---
color_map = {"Tinggi": "red", "Rendah": "pink"}
legend_patches = [
    Patch(facecolor="red", edgecolor="black", label="Tinggi"),
    Patch(facecolor="pink", edgecolor="black", label="Rendah")
]

# --------------- PLOT 1: kelas_1 (K-Means) ---------------
fig1, ax1 = plt.subplots(figsize=(13,8))
gdf_java.plot(
    ax=ax1,
    color=gdf_java["kelas_1"].map(color_map),
    edgecolor="black", linewidth=0.8
)
for _, r in gdf_java.iterrows():
    pt = r.geometry.representative_point()
    ax1.text(pt.x, pt.y, r["Provinsi"], ha="center", va="center", fontsize=8, weight="bold")
ax1.set_axis_off()
ax1.set_title("Peta Hasil Clustering (K-Means Clustering)", fontsize=13, weight="bold")
ax1.legend(handles=legend_patches, loc="lower left", frameon=True, title="Kategori")
plt.tight_layout()
plt.show()

In [ ]:
# --------------- PLOT 2: kelas_2 (K-Medoids) ---------------
fig2, ax2 = plt.subplots(figsize=(13,8))
gdf_java.plot(
    ax=ax2,
    color=gdf_java["kelas_2"].map(color_map),
    edgecolor="black", linewidth=0.8
)
for _, r in gdf_java.iterrows():
    pt = r.geometry.representative_point()
    ax2.text(pt.x, pt.y, r["Provinsi"], ha="center", va="center", fontsize=8, weight="bold")
ax2.set_axis_off()
ax2.set_title("Peta Hasil Clustering (K-Medoids Clustering)", fontsize=13, weight="bold")
ax2.legend(handles=legend_patches, loc="lower left", frameon=True, title="Kategori")
plt.tight_layout()
plt.show()